In [61]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import time
import math

In [62]:
# Global Variables

ratedCap = 8 #MW
costPerMW = 0.9 #£mil per MW
initElecPrice = 55 #£/mwh 
capFactor = 0.3 #Output is 30% of RC
availability = 0.95 #Assume turbines are on 95% of the time
numberOfTurbines=100

inflationRate = 1.05
riskAdjRate = 0.07 #risk-adjusted discount rate
riskFreeRate = 0.03

assetLife = 20
gearBoxLife = 10



impCostGearbox = 100 # millions
impCostGenarator = 150
abdValue = 500

In [63]:
# Legacy Function Definitions

def findSetupCost(ratedCap, numberOfTurbines, costPerMW):
    setupCost = numberOfTurbines * costPerMW * ratedCap
    return setupCost

def netYieldCalculation(numberOfTurbines, ratedCap, capFactor, availability):
    grossYield = numberOfTurbines * ratedCap * capFactor * 8760 #8760 hours in a year
    netYield = availability * grossYield
    return netYield

#assetLife, initElecPrice, inflationRate,riskAdjRate,numberOfTurbines
def cashFlowCalculation(*args):
    discountedCashFlows = np.zeros(assetLife)

    
    
    for i in range(0,assetLife):
        netYield = netYieldCalculation(numberOfTurbines, ratedCap, capFactor, availability)
        discountedCashFlows[i] = pow(10,-6)* 0.7 * np.random.normal(initElecPrice,3) * pow(inflationRate,i) * netYield / pow((1 + riskAdjRate), i)
        
    return discountedCashFlows


#assetLife, initElecPrice, inflationRate,riskAdjRate,numberOfTurbines
def baseCashFlowCalculation(*args):
    discountedBaseCashFlows = np.zeros(assetLife)

    
    
    for i in range(0,assetLife):
        netYield = netYieldCalculation(numberOfTurbines, ratedCap, capFactor, availability)
        discountedBaseCashFlows[i] = pow(10,-6)* 0.7 * initElecPrice * pow(inflationRate,i) * netYield / pow((1 + riskAdjRate), i)
        
    return discountedBaseCashFlows

In [64]:
# Simulation Environment Functions

def TurbineDeathSimulation(scale, loc):
    """Takes the baseCashFlows and simulates how turbine failures would change overall project npv"""
    
    discountedBaseCashFlows = baseCashFlowCalculation()
    x= np.floor(np.random.normal(loc = loc , scale = scale , size = numberOfTurbines)).astype('int')
    unique, counts = np.unique(x, return_counts=True)
    y = np.asarray((unique, 1 - np.cumsum(counts)/numberOfTurbines)).T

    if np.any(y<0):
        y = np.asarray([ [ i[0]-min(y[:,0]) , i[1] ] for i in y ])
        
        if np.any(y>assetLife-1):
            y = y[y[:,0] < assetLife]
            y[-1][1] = 0

    elif np.any(y>assetLife-1):
        y = np.asarray([ [ i[0] - ( max(y[:,0]) - (assetLife-1) ) , i[1] ] for i in y ]) 

        if np.any(y<0):
            startValue = y[0][1]
            y = y[y[:,0] > -1]
            y[0][1] = startValue
            
            
    
    probArray = np.ones(assetLife)
    for count,i in enumerate(y):
        probArray[i[0].astype('int')] = i[1]
    probArray = [0 if count>y[-1][0] else i for count,i in enumerate(probArray)]    
    
    output = discountedBaseCashFlows * probArray
    
    return output[output > 0]
    

    
    
def SigmaCalculator(testFunction,arguments):
    """Takes the price fluctuated cash flows and calculates the mean of the standard deviation of the
    logrithemic returns ,from it"""
    
    sigma_array = []
    for _ in range(0,500):
        
        discountedCashFlows = testFunction(arguments)
        logrithmicReturns = np.zeros(len(discountedCashFlows) - 1)
        
        for g in enumerate(discountedCashFlows):
            if g[0]<len(discountedCashFlows) - 1:
                logrithmicReturns[g[0]] = np.log( discountedCashFlows[g[0]+1]/discountedCashFlows[g[0]] )

        sigma_array.append(np.std(logrithmicReturns))

    sigma = np.mean(sigma_array)
    return sigma
       
def substitute(multiTreeOption):
    return multiTreeOption  

In [65]:
# Option Valuation Functions
def GearboxVal(tree , steps , p , delta , assetLife , impCostGearbox):
    """ Gives us the npv of the gear box replacement option applied """   

    count = 0
    optionTree = np.copy(tree)

    for i in range(0,steps):
        
            case0 = GeneratorVal(assetLife,steps,optionTree[i,steps-1])
            
            #if case0 > optionTree[i,steps-1]:
                #print("hello")
            optionTree[i,steps-1] = max(optionTree[i,steps-1] ,case0[0,0] )
                                                                                  
    emul = np.e**(-1* riskFreeRate * delta) 
    for i in range(steps-1, 0 ,-1):
        for j in range(0,steps-1):
            case1 = ( p*optionTree[j , i] + (1-p)*optionTree[j+1 , i] ) * emul 
            optionTree[j , i-1] = optionTree[j , i-1]/optionTree[j , i-1] * case1


    return optionTree




def AbandonmentVal(optionTree , steps , p , delta , abdValue):
    """ Gives us the npv of the Abandonment option applied """

    option_values = np.copy(optionTree)

    for i in range(0,steps):
        option_values[i , steps-1] = max( option_values[i , steps-1] , abdValue )
    
    emul = np.e**(-1* riskFreeRate * delta)
    for i in range(steps-1, 0 ,-1):
        for j in range(0,steps-1):
            case1 = ( p*option_values[j , i] + (1-p)*option_values[j+1 , i] ) * emul
            case2 = (option_values[j , i -1] / option_values[j , i -1] ) * abdValue

            option_values[j , i -1 ] = max(case1 , case2)


    return option_values    
        

    

    
def GeneratorVal(assetLife,steps,currentV):
    
    Gentree = np.zeros((steps, steps))
    Gentree[Gentree == 0] = None
    
    dates = [i if i>0 else 0 for i in np.sort(np.random.normal(loc = 5 , scale = 2.5 , size = steps))] # Gen Death sim
    oApplied = [(currentV * (pow(inflationRate,assetLife+i) - 1 ) / ( pow(inflationRate,assetLife) - 1))-impCostGearbox  for i in dates]
    
    genSigma = SigmaCalculator(substitute , (np.absolute(oApplied))) #calc sigma before option application
   
    for count,dd in enumerate(oApplied):
        oApplied[count] = max(oApplied[count] , (currentV * (pow(inflationRate,assetLife+gearBoxLife) - 1 ) / ( pow(inflationRate,assetLife) - 1)) - impCostGenarator - impCostGearbox)
    
   
    
    
    delta = max(dates) / steps 
    u = np.exp(genSigma * np.sqrt(delta)) 
    d = 1/u
    p = ( np.exp(riskFreeRate * delta) - d ) / (u - d)
    if p>1:
        p = 0.6 # takes average if p > 1
    if math.isnan(genSigma) :
        p = 0.6
    
    
    Gentree[:,-1] = np.fliplr([oApplied])
   
    
   # back propagation algorithem
    for i in range(steps-1, 0 ,-1):
        for j in range(0,steps-1):

            case1 = ( p*Gentree[j , i] + (1-p)*Gentree[j+1 , i] )           
            Gentree[j , i-1] = case1
            
    return Gentree
    
    
                                      
                                      
def InitCalc(cashFlows ,steps , impCostGearbox , abdValue):
    """ Brings together all the other option calculations """
    
    S0 = sum(cashFlows)
    assetLife = len(cashFlows)
    delta = assetLife / steps #asset life is cut
    u = np.exp(sigma * np.sqrt(delta)) 
    d = 1/u
    p = ( np.exp(riskFreeRate * delta) - d ) / (u - d)
    tree = np.zeros((steps, steps))
    

    #Tree Initial Seeding  
    for i in range(0,steps):
        tree[0 , i] = S0 * pow(u,i)  


    #Tree Propagator     
    for i in range(0,steps-1):
        if i < steps-1:
            for j in range(0,steps):
                if j < steps-1:
                    tree[i+1 , j+1] = tree[i,j] * d
    tree[tree == 0] = None


    optionTree = GearboxVal(tree , steps , p , delta , assetLife , impCostGearbox)
    AoptionTree = AbandonmentVal(optionTree , steps , p , delta , abdValue)

    return tree , optionTree , AoptionTree




In [66]:
# Testing single failure scenario trees

#sigma = SigmaCalculator(cashFlowCalculation,())
sigma = 0.085
#TurbineDeathSimulation(scale = 1,loc = 10)
tree,optionTree,AoptionTree = InitCalc(TurbineDeathSimulation(scale = 1,loc = 10) ,steps = 15 , impCostGearbox = impCostGearbox , abdValue = abdValue)

#optionTree[0,0] - tree[0,0] # For option value
#pd.DataFrame(tree) # Base tree depending on failure scenario
#pd.DataFrame(optionTree) # Base tree depending on failure scenario , with gearbox replacement option applied
#pd.DataFrame(AoptionTree) # Base tree depending on failure scenario , with gearbox replacement + Abandonment option applied

In [67]:
pd.DataFrame(AoptionTree)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1465.244269,1590.238877,1725.327958,1871.334712,2029.158134,2199.773240,2384.227294,2583.633919,2799.170154,3032.083282,3283.711759,3555.515305,3849.094045,4166.171950,4508.612750
1,NaN,1341.482863,1456.932880,1581.692341,1716.504977,1862.195047,2019.674687,2189.943663,2374.077831,2573.208576,2788.504971,3021.179794,3272.541164,3544.076602,3837.377492
2,NaN,NaN,1226.369283,1333.028872,1448.301460,1572.837512,1707.360362,1852.687599,2009.750940,2179.599974,2363.375377,2562.247337,2777.340657,3009.732108,3260.824611
3,NaN,NaN,NaN,1119.143773,1217.660175,1324.212299,1439.364689,1563.721570,1697.952361,1842.841656,1999.347630,2168.635450,2352.035278,2550.833561,2765.598504
4,NaN,NaN,NaN,NaN,1019.185810,1110.037602,1208.461970,1314.990886,1430.153209,1554.458313,1688.434427,1832.735543,1988.311491,2156.660246,2340.435399
5,NaN,NaN,NaN,NaN,NaN,926.133554,1009.628237,1100.259098,1198.599620,1305.249133,1420.717826,1545.345477,1679.302968,1822.627314,1974.857958
6,NaN,NaN,NaN,NaN,NaN,NaN,839.905647,916.323819,999.338294,1089.515247,1187.653393,1294.601147,1411.022290,1537.337860,1674.474245
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,760.463171,830.288840,906.147473,988.229032,1077.297628,1174.588109,1281.241784,1396.807410
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,687.402030,751.193732,821.087551,896.262923,976.696558,1063.873489,1161.850724
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,620.157969,677.384642,742.588415,813.472211,887.322729,961.141072


In [68]:
pd.DataFrame(tree)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,678.541137,732.139755,789.972180,852.372843,919.702595,992.350789,1070.737534,1155.316123,1246.575657,1345.043869,1451.290180,1565.928989,1689.623229,1823.088195,1967.095688
1,NaN,628.866376,678.541137,732.139755,789.972180,852.372843,919.702595,992.350789,1070.737534,1155.316123,1246.575657,1345.043869,1451.290180,1565.928989,1689.623229
2,NaN,NaN,582.828214,628.866376,678.541137,732.139755,789.972180,852.372843,919.702595,992.350789,1070.737534,1155.316123,1246.575657,1345.043869,1451.290180
3,NaN,NaN,NaN,540.160422,582.828214,628.866376,678.541137,732.139755,789.972180,852.372843,919.702595,992.350789,1070.737534,1155.316123,1246.575657
4,NaN,NaN,NaN,NaN,500.616261,540.160422,582.828214,628.866376,678.541137,732.139755,789.972180,852.372843,919.702595,992.350789,1070.737534
5,NaN,NaN,NaN,NaN,NaN,463.967056,500.616261,540.160422,582.828214,628.866376,678.541137,732.139755,789.972180,852.372843,919.702595
6,NaN,NaN,NaN,NaN,NaN,NaN,430.000873,463.967056,500.616261,540.160422,582.828214,628.866376,678.541137,732.139755,789.972180
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,398.521292,430.000873,463.967056,500.616261,540.160422,582.828214,628.866376,678.541137
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.346274,398.521292,430.000873,463.967056,500.616261,540.160422,582.828214
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,342.307106,369.346274,398.521292,430.000873,463.967056,500.616261


In [69]:
# Below here is for running the american option
#################################################################################

In [70]:
# ######################
# calculates out option values for different failure situations and stores in multiTreeOption array
def calc_price(initElecPrice):
    multiTreeOption = []
    steps = 15
    locVal = np.arange(1, assetLife-1, (assetLife-2) / steps )

    for n in range(0,len(locVal)): 

        averageOptionValue=[]

        for _ in range(0,30):    
            tree,optionTree,AoptionTree = InitCalc(TurbineDeathSimulation(scale = 1,loc = locVal[n]) ,steps = 15 ,
                                                   impCostGearbox = impCostGearbox , abdValue = abdValue)        
            averageOptionValue.append(AoptionTree[0,0]-tree[0,0])

        multiTreeOption.append( np.mean(averageOptionValue) )

    multiTreeOption = np.asarray(multiTreeOption)

    # ######################
    # Estimating mean of standard deviation of logrithmic returns of multiTreeOption


    metaSigma = SigmaCalculator(substitute , (multiTreeOption))


    print("mean of logrithmic returns standard deviations is: " + str(metaSigma))



    # ######################
    # Final tree seeding
    print("assetlife",assetLife)    
    delta = assetLife / steps 
    u = np.exp(metaSigma * np.sqrt(delta)) 
    d = 1/u
    p = ( np.exp(riskFreeRate * delta) - d ) / (u - d)

    finalTree = np.zeros((steps, steps))

    if multiTreeOption[0] < multiTreeOption[-1]:  # When seeding largest values must be first as that is how BT works
        finalTree[:,-1] = np.fliplr([multiTreeOption])
    else:
        finalTree[:,-1] = multiTreeOption



    finalTree[finalTree == 0] = None

    #back propagation algorithem
    for i in range(steps-1, 0 ,-1):
        for j in range(0,steps-1):

            case1 = ( p*finalTree[j , i] + (1-p)*finalTree[j+1 , i] )           
            finalTree[j , i-1] = case1

    #print(metaSigma ,delta, u,d,p)         
    #pd.DataFrame(finalTree)
    return finalTree[0,0]
    

In [55]:
Am_option_value = calc_price(initElecPrice)
setupCost = findSetupCost(ratedCap, numberOfTurbines, costPerMW)
BAUNPV = tree[0,0]-setupCost
Am_project_value = Am_option_value - setupCost

print("Setup Cost = £%.2f million" %setupCost)
print("BAU Net Present Value = £%.2f million" %BAUNPV)
print("Value of option = £%.2f million" %Am_option_value)
print("Value of project = £%.2f million" %Am_project_value)


mean of logrithmic returns standard deviations is: 0.05323064283023252
assetlife 20
Setup Cost = £720.00 million
BAU Net Present Value = £-52.12 million
Value of option = £935.36 million
Value of project = £215.36 million
